In [ ]:
# Receiver Block 1: Install and Import Libraries
!pip install cryptography
from cryptography.hazmat.primitives import hashes, padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.asymmetric import rsa, padding as asym_padding
from cryptography.hazmat.primitives import serialization
import os
from IPython.display import Image, display  # For displaying images in Colab

print("Libraries installed and imported successfully!")

Libraries installed and imported successfully!


In [ ]:
# Receiver Block 2: Generate and Save RSA Keys
def generate_rsa_keys():
    try:
        private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
        public_key = private_key.public_key()
        return private_key, public_key
    except Exception as e:
        print(f"Error generating RSA keys: {e}")
        return None, None

def save_keys(private_key, public_key):
    try:
        with open("/content/private_key.pem", "wb") as f:
            f.write(private_key.private_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PrivateFormat.PKCS8,
                encryption_algorithm=serialization.NoEncryption()
            ))
        with open("/content/public_key.pem", "wb") as f:
            f.write(public_key.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            ))
        print("RSA keys saved in /content/. Share public_key.pem with the sender!")
    except Exception as e:
        print(f"Error saving keys: {e}")

# Generate keys (run this once if starting fresh)
print("Run this to generate RSA keys if you don’t have them yet. Skip if you already have keys.")
generate_keys = input("Generate RSA keys? (yes/no): ").strip().lower()
if generate_keys == "yes":
    private_key, public_key = generate_rsa_keys()
    if private_key and public_key:
        save_keys(private_key, public_key)

Run this to generate RSA keys if you don’t have them yet. Skip if you already have keys.
Generate RSA keys? (yes/no): yes
RSA keys saved in /content/. Share public_key.pem with the sender!


In [ ]:
# Receiver Block 3: Load RSA Private Key
def load_private_key(private_key_path):
    try:
        with open(private_key_path, "rb") as f:
            return serialization.load_pem_private_key(f.read(), password=None)
    except Exception as e:
        print(f"Error loading private key: {e}")
        return None

print("Private key loading function defined. Provide the path in the next step.")

Private key loading function defined. Provide the path in the next step.


In [ ]:
# Receiver Block 4: Decryption Function
def decrypt_file(encrypted_file, output_file, private_key):
    try:
        with open(encrypted_file, "rb") as f:
            data = f.read()

        encrypted_aes_key = data[:256]
        encrypted_camellia_key = data[256:512]
        aes_iv = data[512:528]
        camellia_iv = data[528:544]
        aes_length = int.from_bytes(data[544:548], byteorder='big')
        camellia_length = int.from_bytes(data[548:552], byteorder='big')
        aes_ciphertext = data[552:552 + aes_length]
        camellia_ciphertext = data[552 + aes_length:552 + aes_length + camellia_length]

        aes_key = private_key.decrypt(
            encrypted_aes_key,
            asym_padding.OAEP(mgf=asym_padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
        )
        camellia_key = private_key.decrypt(
            encrypted_camellia_key,
            asym_padding.OAEP(mgf=asym_padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
        )

        aes_cipher = Cipher(algorithms.AES(aes_key), modes.CBC(aes_iv))
        aes_decryptor = aes_cipher.decryptor()
        padded_part1 = aes_decryptor.update(aes_ciphertext) + aes_decryptor.finalize()

        camellia_cipher = Cipher(algorithms.Camellia(camellia_key), modes.CBC(camellia_iv))
        camellia_decryptor = camellia_cipher.decryptor()
        padded_part2 = camellia_decryptor.update(camellia_ciphertext) + camellia_decryptor.finalize()

        unpadder1 = padding.PKCS7(128).unpadder()
        part1 = unpadder1.update(padded_part1) + unpadder1.finalize()
        unpadder2 = padding.PKCS7(128).unpadder()
        part2 = unpadder2.update(padded_part2) + unpadder2.finalize()

        plaintext = part1 + part2
        with open(output_file, "wb") as f:
            f.write(plaintext)

        print(f"File decrypted and saved as {output_file}")
        # Display based on file type
        if output_file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):
            print("Displaying decrypted image:")
            display(Image(filename=output_file))
        elif output_file.lower().endswith('.txt'):
            print("Displaying decrypted text:")
            with open(output_file, "r") as f:
                print(f.read())
    except FileNotFoundError:
        print(f"Error: Encrypted file '{encrypted_file}' not found.")
    except Exception as e:
        print(f"Error during decryption: {e}")

In [ ]:
# Receiver Block 5: Receiver’s Main Execution
# Upload encrypted_file.bin and private_key.pem to /content/
encrypted_file = input("Enter encrypted file path (e.g., /content/encrypted_file.bin): ").strip()
output_file = input("Enter decrypted output file path (e.g., /content/decrypted_text.txt or /content/decrypted_image.jpg): ").strip()
private_key_path = input("Enter RSA private key file path (e.g., /content/private_key.pem): ").strip()

private_key = load_private_key(private_key_path)
if private_key:
    decrypt_file(encrypted_file, output_file, private_key)
else:
    print("Decryption aborted due to private key loading failure.")

KeyboardInterrupt: Interrupted by user